# Lab: Amazon SageMaker에서 자체 스크립트 가져오기

<div class="alert alert-block alert-info">
⚠️ 이 노트북과 호환되는 것으로 알려진 최신 SageMaker Distribution 이미지 버전은 <code>3.1.0</code>입니다. 다른 버전에서 문제가 발생하면 버전 <code>3.1.0</code>으로 다운그레이드하세요. <b>이를 위해서는 JupyterApp을 중지하고, SageMaker Distribution 이미지를 <code>3.1.0</code>으로 다운그레이드한 다음, 변경 사항을 적용하기 위해 JupyterLabApp을 다시 시작해야 합니다</b>.</div>

## TensorFlow 스크립트 모드 훈련 및 서빙
스크립트 모드는 TensorFlow 훈련 스크립트 형식으로, 최소한의 수정으로 SageMaker에서 모든 TensorFlow 훈련 스크립트를 실행할 수 있게 해줍니다. [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk)는 스크립트를 SageMaker 훈련 인스턴스로 전송하는 작업을 처리합니다. 훈련 인스턴스에서 SageMaker의 기본 TensorFlow 지원은 훈련 관련 환경 변수를 설정하고 훈련 스크립트를 실행합니다. 이 튜토리얼에서는 SageMaker Python SDK를 사용하여 훈련 작업을 시작하고 훈련된 모델을 배포합니다.

스크립트 모드는 Python 스크립트, Python 모듈 또는 셸 스크립트를 사용한 훈련을 지원합니다. 이 예제에서는 Python 스크립트를 사용하여 [MNIST 데이터셋](http://yann.lecun.com/exdb/mnist/)에서 분류 모델을 훈련합니다. 이 예제에서는 SageMaker Python SDK를 사용하여 `TensorFlow 2.1.x` 스크립트로 SageMaker에서 모델을 훈련하는 방법을 보여드리겠습니다. 또한, 이 노트북은 [SageMaker TensorFlow Serving 컨테이너](https://github.com/aws/sagemaker-tensorflow-serving-container)를 사용하여 실시간 추론을 수행하는 방법을 보여줍니다. TensorFlow Serving 컨테이너는 스크립트 모드의 기본 추론 방법입니다. TensorFlow Serving 컨테이너에 대한 전체 문서는 [여기](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/deploying_tensorflow_serving.rst)를 방문하세요.

이 노트북에서는 TensorFlow 패키지가 호출되지 않으므로, 계속 진행하려면 `Python 3 (ipykernel)` 커널을 선택하세요.

# 환경 설정하기
환경 설정부터 시작하겠습니다:

In [ ]:
# cell 01
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

# 학습 데이터
MNIST 데이터셋은 `sagemaker-sample-data-<REGION>` 공개 S3 버킷의 `tensorflow/mnist` 접두사 아래에 로드되어 있습니다. 이 접두사 아래에는 네 개의 .npy 파일이 있습니다:

- train_data.npy
- eval_data.npy
- train_labels.npy
- eval_labels.npy

In [ ]:
# cell 02
training_data_uri = 's3://sagemaker-sample-data-{}/tensorflow/mnist'.format(region)

# 분산 학습을 위한 스크립트 구성하기
이 튜토리얼의 학습 스크립트는 TensorFlow의 공식 [CNN MNIST 예제](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/layers/cnn_mnist.py)에서 수정되었습니다. SageMaker에서 전달하는 `model_dir` 매개변수를 처리하도록 수정했습니다. 이는 분산 학습 중 데이터 공유와 체크포인팅 및/또는 모델 지속성을 위해 사용할 수 있는 S3 경로입니다. 또한 학습 관련 변수 처리를 위한 인수 파싱 함수도 추가했습니다.

학습 작업이 끝날 때 환경 변수 `SM_MODEL_DIR`에 저장된 경로(항상 `/opt/ml/model`을 가리킴)로 학습된 모델을 내보내는 단계를 추가했습니다. 이는 SageMaker가 학습 종료 시 이 폴더의 모든 모델 아티팩트를 S3에 업로드하기 때문에 매우 중요합니다.

다음은 전체 스크립트입니다:

In [ ]:
# cell 03

# TensorFlow 2.1 script
!pygmentize 'mnist-2.py'

# TensorFlow 추정기를 사용하여 훈련 작업 생성하기
`sagemaker.tensorflow.TensorFlow` 추정기는 스크립트 모드 컨테이너 위치 지정, 스크립트를 S3 위치에 업로드, SageMaker 훈련 작업 생성을 처리합니다. 여기서 몇 가지 중요한 매개변수를 살펴보겠습니다:

`py_version`은 `'py3'`로 설정하여 스크립트 모드를 사용함을 나타냅니다. 레거시 모드는 `Python 2`만 지원하기 때문입니다. 레거시 모드에서 `Python 2`로 `Tensorflow`를 실행하는 것은 권장하지 않습니다.

`distribution`은 분산 훈련 설정을 구성하는 데 사용됩니다. 이는 여러 인스턴스나 여러 GPU에 걸쳐 분산 훈련을 수행하는 경우에만 필요합니다. 여기서는 파라미터 서버를 분산 훈련 스키마로 사용합니다. SageMaker 훈련 작업은 동종 클러스터에서 실행됩니다. SageMaker 설정에서 파라미터 서버의 성능을 향상시키기 위해, 클러스터의 모든 인스턴스에서 파라미터 서버를 실행하므로 시작할 파라미터 서버 수를 지정할 필요가 없습니다. 스크립트 모드는 [Horovod](https://github.com/horovod/horovod)를 사용한 분산 훈련도 지원합니다. 분산 구성 방법에 대한 전체 문서는 [여기](https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/tensorflow#distributed-training)에서 확인할 수 있습니다.

`instance_type`은 훈련에 사용되는 EC2 인스턴스를 지정합니다. 데이터 크기, 알고리즘 및 작업에 따라 적절한 훈련 인스턴스를 선택해야 합니다. 여기서는 `ml.c5.xlarge`를 선택합니다. NVIDIA T4 GPU와 커스텀 Intel Cascade Lake CPU를 특징으로 하며 머신 러닝 추론 및 소규모 훈련에 최적화된 [G4dn](https://aws.amazon.com/ec2/instance-types/g4/) 인스턴스에 대해서도 자세히 알아볼 수 있습니다. [사용 가능한 인스턴스 유형 및 가격](https://aws.amazon.com/sagemaker/pricing/)에 대해 더 자세히 알아보세요.

`use_spot_instances`(선택 사항): 비용 최적화를 위해 이 매개변수를 `True`로 설정하여 [관리형 Amazon EC2 스팟 인스턴스](https://docs.aws.amazon.com/sagemaker/latest/dg/model-managed-spot-training.html)를 활용할 수 있습니다. 관리형 스팟 훈련은 온디맨드 인스턴스 대비 최대 90%까지 모델 훈련 비용을 최적화할 수 있습니다. SageMaker가 스팟 인터럽트를 대신 관리합니다. 어떤 훈련 작업이 스팟 인스턴스를 사용할지 지정하고, Amazon SageMaker가 Amazon EC2 스팟 인스턴스를 사용하여 작업을 실행하기 위해 대기하는 시간을 지정하는 중지 조건을 설정할 수 있습니다. 전체 문서는 [여기](https://sagemaker-examples.readthedocs.io/en/latest/sagemaker-python-sdk/managed_spot_training_tensorflow_estimator/managed_spot_training_tensorflow_estimator.html)에서 확인할 수 있습니다.

`TensorFlow` `2.1` 스크립트로 훈련하기 위해 추정기를 초기화할 수 있으며, 올바른 `framework_version`, 즉 `2.1.0`을 지정해야 합니다.

In [ ]:
# cell 04

from sagemaker.tensorflow import TensorFlow

mnist_estimator = TensorFlow(
    entry_point='mnist-2.py',
    role=role,
    instance_count=2,
    instance_type='ml.c5.xlarge',
    framework_version='2.1.0',
    py_version='py3',
    distribution={'parameter_server': {'enabled': True}}
)

# `fit` 호출하기
훈련 작업을 시작하기 위해 `estimator.fit(training_data_uri)`를 호출합니다.

여기서는 S3 위치가 입력으로 사용됩니다. fit은 이 S3 위치를 가리키는 'training'이라는 기본 채널을 생성합니다. 훈련 스크립트에서는 SM_CHANNEL_TRAINING에 저장된 위치에서 훈련 데이터에 접근할 수 있습니다. fit은 다른 유형의 입력도 몇 가지 허용합니다. 자세한 내용은 [여기](https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.EstimatorBase.fit)에서 API 문서를 참조하세요.

훈련이 시작되면 `TensorFlow` 컨테이너는 mnist.py를 실행하고, estimator에서 하이퍼파라미터와 model_dir을 스크립트 인수로 전달합니다. 이 예제에서는 둘 다 정의하지 않았기 때문에 하이퍼파라미터는 전달되지 않으며, model_dir은 기본값인 `s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>`으로 설정됩니다. 따라서 스크립트 실행은 다음과 같습니다:

`python mnist-2.py --model_dir s3://<DEFAULT_BUCKET>/<TRAINING_JOB_NAME>`

훈련이 완료되면 훈련 작업은 TensorFlow 서빙을 위한 저장된 모델을 업로드합니다.

TensorFlow 2.1 스크립트로 모델을 훈련하기 위해 fit 호출하기.

In [ ]:
# cell 05
mnist_estimator.fit(training_data_uri)

# 학습된 모델을 엔드포인트에 배포하기
`deploy()` 메서드는 SageMaker 모델을 생성하고, 이를 엔드포인트에 배포하여 실시간으로 예측 요청을 처리합니다. 스크립트 모드로 학습했기 때문에 엔드포인트에는 TensorFlow Serving 컨테이너를 사용할 것입니다. 이 서빙 컨테이너는 SageMaker 호스팅 프로토콜과 호환되는 웹 서버 구현을 실행합니다. [자체 추론 코드 사용하기](https://render.githubusercontent.com/view/ipynb?color_mode=auto&commit=a5c9a21e6ed70fd51ab5178f3a35461473f7b379&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f6177732f616d617a6f6e2d736167656d616b65722d6578616d706c65732f613563396132316536656437306664353161623531373866336133353436313437336637623337392f736167656d616b65722d707974686f6e2d73646b2f74656e736f72666c6f775f7363726970745f6d6f64655f747261696e696e675f616e645f73657276696e672f74656e736f72666c6f775f7363726970745f6d6f64655f747261696e696e675f616e645f73657276696e672e6970796e62&nwo=aws%2Famazon-sagemaker-examples&path=sagemaker-python-sdk%2Ftensorflow_script_mode_training_and_serving%2Ftensorflow_script_mode_training_and_serving.ipynb&repository_id=107937815&repository_type=Repository) 문서는 SageMaker가 추론 컨테이너를 실행하는 방법을 설명합니다.

In [ ]:
# cell 06
predictor = mnist_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

# 엔드포인트 호출하기
학습 데이터를 다운로드하고 이를 추론을 위한 입력으로 사용해 보겠습니다.

In [ ]:
# cell 07
import numpy as np

!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_data.npy train_data.npy
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/train_labels.npy train_labels.npy

train_data = np.load('train_data.npy')
train_labels = np.load('train_labels.npy')

입력 및 출력 데이터의 형식은 [TensorFlow Serving REST API](https://www.tensorflow.org/serving/api_rest)의 Predict 메서드의 요청 및 응답 형식과 직접 대응됩니다. SageMaker의 TensforFlow Serving 엔드포인트는 TensorFlow REST API의 일부가 아닌 추가 입력 형식도 허용할 수 있으며, 여기에는 간소화된 JSON 형식, 줄 구분 JSON 객체("jsons" 또는 "jsonlines") 및 CSV 데이터가 포함됩니다.

이 예제에서는 간소화된 JSON 형식으로 직렬화될 numpy 배열을 입력으로 사용하고 있습니다. 또한, 다음 코드에서 볼 수 있듯이 TensorFlow serving은 여러 항목을 한 번에 처리할 수도 있습니다. TensorFlow serving SageMaker 엔드포인트에 대한 예측을 수행하는 방법에 대한 전체 문서는 [여기](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/tensorflow/deploying_tensorflow_serving.rst#making-predictions-against-a-sagemaker-endpoint)에서 찾을 수 있습니다.

In [ ]:
# cell 08
predictions = predictor.predict(train_data[:50])
for i in range(0, 50):
    prediction = np.argmax(predictions['predictions'][i])
    label = train_labels[i]
    print('prediction is {}, label is {}, matched: {}'.format(prediction, label, prediction == label))

# 엔드포인트 삭제하기
방금 생성한 엔드포인트를 삭제하여 추가 비용이 발생하지 않도록 하고 [확인](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html)해 보겠습니다.

In [ ]:
# cell 09
predictor.delete_endpoint()